In [1]:
import os
import re
import cv2
import numpy as np
from ultralytics import YOLO
from skimage import exposure
from typing import Tuple
import logging

logging.getLogger("ultralytics").setLevel(logging.ERROR)

# ========= VARIÁVEIS DE CONFIGURAÇÃO =========
INPUT_FOLDER = "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/article/analisar"  # ✅ Altere aqui
MODEL_PATH = "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/yolo-train/eggs-scanner-image.v2i.yolov11/runs/detect/train2/weights/best-train2.onnx"  # ✅ Altere aqui
# ============================================

PATTERN = re.compile(r"(.*)\.(png|jpg|jpeg)$", re.IGNORECASE)
WINDOW_SIZE = 254


class ImageProcessor:
    def __init__(self, model: YOLO, window_size: int = WINDOW_SIZE) -> None:
        self.window_size = window_size
        self.model = model
        self.image = None
        self.processed_image = None
        self.object_counts = []

    def load_image(self, image_path: str) -> np.ndarray:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return image

    def normalize_square(self, square: np.ndarray) -> np.ndarray:
        return exposure.adjust_gamma(square, gamma=1.5)

    def predict_and_draw_boxes(self, square: np.ndarray) -> Tuple[int, np.ndarray]:
        results = self.model(square, verbose=False)
        boxes = results[0].boxes
        output = square.copy()

        if boxes is None or boxes.xyxy is None:
            return 0, output

        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            score = float(box.conf[0]) if box.conf is not None else 0.0
            label = f"ovo {score * 100:.2f} (%)"

            # Caixa VERDE
            cv2.rectangle(output, (x1, y1), (x2, y2), (0, 255, 0), 2)

            text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            text_x = x1
            text_y = y1 - 5 if y1 - 5 > 10 else y1 + text_size[1] + 5
            cv2.putText(output, label, (text_x, text_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        return len(boxes), output

    def process_image(self, image_path: str, output_folder: str) -> None:
        self.image = self.load_image(image_path)
        img_height, img_width, channels = self.image.shape
        padded_height = ((img_height + self.window_size - 1) // self.window_size) * self.window_size
        padded_width = ((img_width + self.window_size - 1) // self.window_size) * self.window_size
        padded_image = np.zeros((padded_height, padded_width, channels), dtype=self.image.dtype)
        padded_image[:img_height, :img_width, :] = self.image
        processed_image = np.copy(padded_image)

        for y in range(0, padded_height, self.window_size):
            for x in range(0, padded_width, self.window_size):
                window = padded_image[y:y + self.window_size, x:x + self.window_size]
                processed_window = self.normalize_square(window)
                _, counted_image = self.predict_and_draw_boxes(processed_window)
                processed_image[y:y + self.window_size, x:x + self.window_size] = counted_image

        self.processed_image = processed_image[:img_height, :img_width]

        base_name, ext = os.path.splitext(os.path.basename(image_path))
        output_name = f"{base_name} - IA{ext}"
        output_path = os.path.join(output_folder, output_name)
        cv2.imwrite(output_path, self.processed_image)
        print(f"✅ Processada: {output_name}")


def load_model_safe(model_path: str) -> YOLO:
    try:
        model = YOLO(model_path)
        dummy = np.zeros((256, 256, 3), dtype=np.uint8)
        model(dummy)
        return model
    except Exception as e:
        if model_path.endswith(".onnx"):
            print(f"⚠️ Erro ao carregar modelo ONNX: {e}")
            fallback_path = model_path.replace(".onnx", ".pt")
            print(f"🔁 Tentando carregar fallback: {fallback_path}")
            return YOLO(fallback_path)
        else:
            raise e


def main():
    print(f"📂 Processando imagens em: {INPUT_FOLDER}")
    os.makedirs(INPUT_FOLDER, exist_ok=True)

    model = load_model_safe(MODEL_PATH)
    processor = ImageProcessor(model)

    image_files = [f for f in os.listdir(INPUT_FOLDER) if PATTERN.match(f)]

    for filename in sorted(image_files):
        full_path = os.path.join(INPUT_FOLDER, filename)
        try:
            processor.process_image(full_path, INPUT_FOLDER)
        except Exception as e:
            print(f"❌ Erro ao processar {filename}: {e}")


if __name__ == "__main__":
    main()


📂 Processando imagens em: /media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/article/analisar


2025-05-08 13:31:35.995720317 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


✅ Processada: paleta-1-25 - IA.jpg
✅ Processada: paleta-10-4 - IA.jpg
✅ Processada: paleta-140-5 - IA.jpg
✅ Processada: paleta-146-4 - IA.jpg
✅ Processada: paleta-16-4 - IA.jpg
✅ Processada: paleta-17-8 - IA.jpg
✅ Processada: paleta-2-2 - IA.jpg
✅ Processada: paleta-2-4 - IA.jpg
✅ Processada: paleta-203-1 - IA.jpg
✅ Processada: paleta-268-3 - IA.jpg
✅ Processada: paleta-285-1 - IA.jpg
✅ Processada: paleta-33-12 - IA.jpg
✅ Processada: paleta-340-9 - IA.jpg
✅ Processada: paleta-4-11 - IA.jpg
✅ Processada: paleta-4-5 - IA.jpg
✅ Processada: paleta-5-20 - IA.jpg
✅ Processada: paleta-50-13 - IA.jpg
✅ Processada: paleta-9-137 - IA.jpg
